In [2]:
# Load modules and reset data

from pathlib import Path
from datetime import datetime, timedelta
import re
from config import *
from sending import (
    get_mf4_files,
    decode_and_send,
    send_signal,
)
from utils import *
from asammdf import MDF
from pprint import pprint

In [3]:
# Set up paths and get files

win_home = get_windows_home_path()
can_logs = Path.joinpath(
    win_home,
    "Epiroc",
    "O365 UMR R&D Battery TMS - General",
    "05_Design",
    "Prototype",
    "Testing",
    "4 Mine Testing (Lalor, Hudbay, Snow Lake) - Prototype V2",
    "CAN LOGS",
)
files = []
if can_logs.exists():
    files = get_mf4_files(can_logs)
    print(f"Found {len(files)} files to process")
else:
    print(f"Cannot find CAN LOGS folder at {can_logs}")

Found 1460 files to process


In [4]:
with open("filtered_paths.txt", "r") as f:
    filtered = [Path(line.strip()) for line in f.readlines()]
print(f"Recovered {len(filtered)} filtered paths")

Recovered 0 filtered paths


In [4]:
# From inspection, there are a lot of duplicate files. Filter them out
# Files will have the same last 3 path parts: ADDRESS/folder/filename.MF4
# The rest of the path is irrelevant, if they have the same last 3 parts, they are duplicates

filtered = list()


def is_duplicate(file: Path) -> bool:
    _fparts = "_".join(file.parts[-3:])
    _fparts = re.sub(r' \(\d+\)', '', _fparts)  # Remove (1), (2), etc.
    for f in filtered:
        if _fparts == "_".join(f.parts[-3:]):
            return True

    return False

def is_raw(file: Path) -> bool:
    if ("Decoded" in str(file)) or ("decoded" in str(file)):
        return False
        
    try:
        with MDF(file) as mdf:
            return "CAN_DataFrame" in mdf.channels_db.keys()
    except Exception as e:
        print(f"Error reading {file}: {e}")
        return False

for file in files:
    if not is_duplicate(file) and is_raw(file):
        filtered.append(file)

print(f"Filtered from {len(files)} to {len(filtered)} files to process")

Error reading C:\Users\CARAL\Epiroc\O365 UMR R&D Battery TMS - General\05_Design\Prototype\Testing\4 Mine Testing (Lalor, Hudbay, Snow Lake) - Prototype V2\CAN LOGS\Field Testing Upload Jan 24 2025\4205 tms trial data\4205 tms trial data\LOG\AE87AB7E\00001087\00000001-Deocded.mf4: File "C:\Users\CARAL\Epiroc\O365 UMR R&D Battery TMS - General\05_Design\Prototype\Testing\4 Mine Testing (Lalor, Hudbay, Snow Lake) - Prototype V2\CAN LOGS\Field Testing Upload Jan 24 2025\4205 tms trial data\4205 tms trial data\LOG\AE87AB7E\00001087\00000001-Deocded.mf4" does not exist
Filtered from 1460 to 1179 files to process


In [6]:
with open("filtered_paths.txt", "w") as f:
    for path in filtered:
        f.write(str(path) + "\n")

In [ ]:
# Select which folder to process
FOLDERS: list[str] = [
    "April 8 Return from Snow Lake CANEdge SD Backup", # 0
    "April 23 CANEdge SD Backup", # 1
    "Archive", # 2
    "Duplicate Uploads", # 3
    "Field Testing", # 4
    "Field Testing Nov 10-18 Decoded", # 5
    "Field Testing Upload Jan 24 2025", # 6
    "Jan28 2025", # 7
    "Prior Testing", # 8
    "TMS LOGS Mar 3 2025", # 9
    "TMS Trial logs feb 25 2025", # 10
]

In [12]:
dbc_file = can_logs.parent.joinpath("dbc","snow_leopard_gen2_windows_no_value_tables.dbc")

if not dbc_file.exists():
    print(f"Cannot find DBC file at {dbc_file}")
    dbc_file = None
database_files = {"CAN": [(dbc_file, 0)]}

MDF_BATCH_SIZE = 50

for folder in [7, 8, 6, 5, 4, 9, 10]:
    selected_folder = FOLDERS[7]
    folder_files = [f for f in filtered if selected_folder in str(f)]
    print(f"Found {len(folder_files)} files in folder {selected_folder} to process")

    for i in range(0, len(folder_files), MDF_BATCH_SIZE):
        cc = MDF()

        _files = []

        if i + MDF_BATCH_SIZE < len(folder_files):
            _files = folder_files[i : i + MDF_BATCH_SIZE]
        else:
            _files = folder_files[i:]

        for file in _files:
            print(f'=> Processing ../{"/".join(file.parts[-3:])}')

        try:
            ts = datetime.now()
            print(f"  ⌛ Concatenating ... ", end="\r", flush=True)
            cc = cc.concatenate(_files)
            print(f"  ☑️ Concatenated in {(datetime.now() - ts).total_seconds():.3f} seconds")

            try:
                ts = datetime.now()
                print(f"  ⌛ Decoding ... ", end="\r", flush=True)
                decoded = cc.extract_bus_logging(database_files=database_files)
                print(f"  ☑️ Decoded in {(datetime.now() - ts).total_seconds():.3f} seconds")

                batch: list[str] = []
                for sig in decoded.iter_channels():
                    send_signal(
                        signal=sig,
                        start_time=decoded.start_time,
                        job="SnowLeopardTMS",
                        print_metric_line=False,
                        send_signal=True,
                        skip_signal_range_check=True
                    )
            except Exception as e:
                print(f"  ❌ Error decoding and sending signals: {e}")
                continue
        except Exception as e:
            print(f"  ❌ Error concatenating files: {e}")
            continue


Found 16 files in folder Jan28 2025 to process
=> Processing ../AE87AB7E/00001099/00000001.MF4
=> Processing ../AE87AB7E/00001099/00000002.MF4
=> Processing ../AE87AB7E/00001100/00000001.MF4
=> Processing ../AE87AB7E/00001100/00000002.MF4
=> Processing ../AE87AB7E/00001101/00000001.MF4
=> Processing ../AE87AB7E/00001102/00000001.MF4
=> Processing ../AE87AB7E/00001103/00000001.MF4
=> Processing ../AE87AB7E/00001104/00000001.MF4
=> Processing ../AE87AB7E/00001105/00000001.MF4
=> Processing ../AE87AB7E/00001106/00000001.MF4
=> Processing ../AE87AB7E/00001107/00000001.MF4
=> Processing ../AE87AB7E/00001108/00000001.MF4
=> Processing ../AE87AB7E/00001109/00000001.MF4
=> Processing ../AE87AB7E/00001110/00000001.MF4
=> Processing ../AE87AB7E/00001111/00000001.MF4
=> Processing ../AE87AB7E/00001112/00000001.MF4
  ☑️ Concatenated in 12.928 seconds
  ☑️ Decoded in 9.756 seconds
  📨 Sending DCDC_Enable [2025-01-23T21:16:57.716650+00:00 - 2025-01-28T15:43:32.552100+00:00, 145035 samples] ... sent 

In [13]:
database_files = {"CAN": [(dbc_file, 0)]}
decoded = MDF(files[4]).extract_bus_logging(database_files=database_files)
pprint([{sig.name: len(sig.timestamps)} for sig in list(decoded.iter_channels())[:30]])

[{'DCDC_Output_Voltage': 8064},
 {'DCDC_Output_Current': 8064},
 {'DCDC_Input_Voltage': 8064},
 {'DCDC_Temperature': 8064},
 {'DCDC_State': 8064},
 {'DCDC_Fault_Level': 8064},
 {'DCDC_Input_OverVolt': 8064},
 {'DCDC_Input_UnderVolt': 8064},
 {'DCDC_Output_OverVolt': 8064},
 {'DCDC_Output_Short': 8064},
 {'DCDC_Output_UnderVolt': 8064},
 {'DCDC_OverTemp': 8064},
 {'DCDC_Output_OverCurr': 8064},
 {'DCDC_SciCommFault': 8064},
 {'DCDC_HwFault': 8064},
 {'DCDC_CanCommFault': 8064},
 {'DCDC_BoostFault': 8064},
 {'DCDC_Hw_Version': 8064},
 {'DCDC_Sw_Version': 8064},
 {'HVH_MaxPower': 3867},
 {'HVH_TargetCoolantTemp': 3867},
 {'HVH_HeaterEnable': 3867},
 {'HVH_ActiveDischarge': 3867},
 {'MotorSpeed': 1598},
 {'PowerHold': 1598},
 {'OnOffDirection': 1598},
 {'MotorSpeed': 1598},
 {'PowerHold': 1598},
 {'OnOffDirection': 1598},
 {'HVH_Status': 806}]


In [14]:
for sig in decoded.iter_channels():
    send_signal(
        signal=sig,
        start_time=decoded.start_time,
        job="SnowLeopardTMS",
        print_metric_line=True,
        send_signal=False,
    )

  ☑️ No new data for DCDC_Output_Voltage, skipping ...
  📨 Sending DCDC_Output_Current [2024-11-16T20:49:40.370650+00:00 - 2024-11-16T21:03:06.677650+00:00, 8064 samples] ... sent in 0.167s   
  📨 Sending DCDC_Input_Voltage [2024-11-16T20:49:40.370650+00:00 - 2024-11-16T21:03:06.677650+00:00, 8064 samples] ... sent in 0.160s   
  📨 Sending DCDC_Temperature [2024-11-16T20:49:40.370650+00:00 - 2024-11-16T21:03:06.677650+00:00, 8064 samples] ... sent in 0.196s   
  📨 Sending DCDC_State [2024-11-16T20:49:40.370950+00:00 - 2024-11-16T21:03:06.677950+00:00, 8064 samples] ... sent in 0.160s   
  📨 Sending DCDC_Fault_Level [2024-11-16T20:49:40.370950+00:00 - 2024-11-16T21:03:06.677950+00:00, 8064 samples] ... sent in 0.144s   
  📨 Sending DCDC_Input_OverVolt [2024-11-16T20:49:40.370950+00:00 - 2024-11-16T21:03:06.677950+00:00, 8064 samples] ... sent in 0.166s   
  📨 Sending DCDC_Input_UnderVolt [2024-11-16T20:49:40.370950+00:00 - 2024-11-16T21:03:06.677950+00:00, 8064 samples] ... sent in 0.20

In [40]:
import time
ts = time.time()
for file in files:
    try:
        with MDF(file, info_only=True) as mdf:
            channels = [ch for ch in mdf.iter_channels() if len(ch.timestamps) > 0]
    except Exception as e:
        print(f"Error reading {file}: {e}")

print(f"Processed {len(files)} files in {time.time() - ts:.2f} seconds")

Error reading C:\Users\CARAL\Epiroc\O365 UMR R&D Battery TMS - General\05_Design\Prototype\Testing\4 Mine Testing (Lalor, Hudbay, Snow Lake) - Prototype V2\CAN LOGS\Field Testing Upload Jan 24 2025\4205 tms trial data\4205 tms trial data\LOG\AE87AB7E\00001076\00000001-Decoded.mf4: File "C:\Users\CARAL\Epiroc\O365 UMR R&D Battery TMS - General\05_Design\Prototype\Testing\4 Mine Testing (Lalor, Hudbay, Snow Lake) - Prototype V2\CAN LOGS\Field Testing Upload Jan 24 2025\4205 tms trial data\4205 tms trial data\LOG\AE87AB7E\00001076\00000001-Decoded.mf4" does not exist
Error reading C:\Users\CARAL\Epiroc\O365 UMR R&D Battery TMS - General\05_Design\Prototype\Testing\4 Mine Testing (Lalor, Hudbay, Snow Lake) - Prototype V2\CAN LOGS\Field Testing Upload Jan 24 2025\4205 tms trial data\4205 tms trial data\LOG\AE87AB7E\00001077\00000001-Decoded.mf4: File "C:\Users\CARAL\Epiroc\O365 UMR R&D Battery TMS - General\05_Design\Prototype\Testing\4 Mine Testing (Lalor, Hudbay, Snow Lake) - Prototype V2

In [41]:
import time
ts = time.time()
for file in files:
    try:
        with MDF(file) as mdf:
            channels = [ch for ch in mdf.iter_channels() if len(ch.timestamps) > 0]
    except Exception as e:
        print(f"Error reading {file}: {e}")

print(f"Processed {len(files)} files in {time.time() - ts:.2f} seconds")

Error reading C:\Users\CARAL\Epiroc\O365 UMR R&D Battery TMS - General\05_Design\Prototype\Testing\4 Mine Testing (Lalor, Hudbay, Snow Lake) - Prototype V2\CAN LOGS\Field Testing Upload Jan 24 2025\4205 tms trial data\4205 tms trial data\LOG\AE87AB7E\00001076\00000001-Decoded.mf4: File "C:\Users\CARAL\Epiroc\O365 UMR R&D Battery TMS - General\05_Design\Prototype\Testing\4 Mine Testing (Lalor, Hudbay, Snow Lake) - Prototype V2\CAN LOGS\Field Testing Upload Jan 24 2025\4205 tms trial data\4205 tms trial data\LOG\AE87AB7E\00001076\00000001-Decoded.mf4" does not exist
Error reading C:\Users\CARAL\Epiroc\O365 UMR R&D Battery TMS - General\05_Design\Prototype\Testing\4 Mine Testing (Lalor, Hudbay, Snow Lake) - Prototype V2\CAN LOGS\Field Testing Upload Jan 24 2025\4205 tms trial data\4205 tms trial data\LOG\AE87AB7E\00001077\00000001-Decoded.mf4: File "C:\Users\CARAL\Epiroc\O365 UMR R&D Battery TMS - General\05_Design\Prototype\Testing\4 Mine Testing (Lalor, Hudbay, Snow Lake) - Prototype V2